In [113]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df_train = pd.read_csv('home_data_kaggle/train.csv')
df_train.info()

In [88]:
y = df_train.SalePrice
X = df_train.drop('SalePrice',axis=1)


In [115]:
X_train_full,X_val_full,y_train,y_val  = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

In [116]:
nulls = [col for col in df_train.columns if df_train[col].isnull().any()]
X_train_full.drop(nulls,axis=1,inplace=True)
X_val_full.drop(nulls,axis=1,inplace=True)



In [118]:
ordinable =[col for col in X_train_full.columns if  X_train_full[col].dtype=='object' and X_train_full[col].nunique()<10]
numerical = [col for col in X_train_full.columns if  X_train_full[col].dtype in['int64','float64']]


usable_cols = ordinable+numerical

X_train = X_train_full[usable_cols] .copy()
X_val = X_val_full[usable_cols].copy()

In [129]:


text_cols = [col for col in X_train.columns if  X_train[col].dtype=='object']

good_cols = [col for col in X_train.columns if set(X_train[col]).issubset(set(X_val[col])) ]
good_cols

bad_cols = list(set(text_cols)-set(good_cols))

X_train_main = X_train.drop(bad_cols,axis=1)
X_val_main = X_val.drop(bad_cols,axis=1)

ordinate_train = X_train_main.copy()
ordinate_val = X_val_main.copy()


In [152]:
(len(ordinate_train.columns),len(ordinate_val.columns))
ordinate_val.index


Index([ 529,  491,  459,  279,  655, 1013, 1403,  601, 1182,  687,
       ...
       1301, 1427, 1402, 1223, 1029,  326,  440, 1387, 1323,   61],
      dtype='int64', length=292)

In [131]:
final_text_cols = [col for col in ordinate_train.columns if  ordinate_train[col].dtype=='object']
len(final_text_cols)


13

In [132]:
my_ordinator = OrdinalEncoder(handle_unknown ='use_encoded_value',unknown_value=-1)

ordinate_train[final_text_cols] =  my_ordinator.fit_transform(X_train_main[final_text_cols])
ordinate_val[final_text_cols] = my_ordinator.transform(X_val_main[final_text_cols])




In [133]:
def encoderFunc(X_train,X_val,y_train,y_val):
    my_mod = RandomForestRegressor()
    my_mod.fit(X_train,y_train)
    pred=my_mod.predict(X_val)
    mae = mean_absolute_error(y_val,pred)
    return mae


encoderFunc(ordinate_train,ordinate_val,y_train,y_val)

17526.752945205477

In [154]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

oh_train = pd.DataFrame(one_hot.fit_transform(ordinate_train[final_text_cols]),index=ordinate_train.index)
oh_val = pd.DataFrame(one_hot.transform(ordinate_val[final_text_cols]),index=ordinate_val.index)


num_oh_train = ordinate_train.drop(final_text_cols,axis=1)
num_oh_val = ordinate_val.drop(final_text_cols,axis=1)



main_oh_train = pd.concat([oh_train,num_oh_train],axis=1)
main_oh_val = pd.concat([oh_val,num_oh_val],axis=1)

main_oh_train.columns = main_oh_train.columns.astype('str')
main_oh_val.columns = main_oh_val.columns.astype('str')


In [156]:
encoderFunc(main_oh_train,main_oh_val,y_train,y_val)

17218.551506849315